# Part 5 : Recommendation sytem, we consolidate all notebook and define a final prediction

To do this notebook I used all previous notebook.  
It's better to read them before going to this one.

# The summary of the notebook is written below

I. Import useful library and python file containing our functions  




In [1]:
%cd ..

/Users/kofficornelis/Documents/Retail_V2


# I. Import useful library and python file containing our functions

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import pandas as pd
import time
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline, KNNWithZScore
from surprise.model_selection import KFold
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
import matplotlib.pyplot as plt
import seaborn as sns
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
import operator
import statistics
from sklearn.metrics import mean_squared_error 
import utils.Function_05 as f5
import utils.Function_02 as f2
import utils.Function_04 as f4

# set the graphs to show in the jupyter notebook
%matplotlib inline

## A. Specify a User for all next prediction

In [3]:
df_non_stand = pd.read_csv('output/non_stand_user_perso_info.csv')
index = False
CURRENT_USER = 266785
customer_index, customer_id = f2.select_customer(index , CURRENT_USER, df_non_stand)
print(f'The index of our specific user is {customer_index} and his Id is {customer_id}')

The index of our specific user is 2 and his Id is 266785


# II. Recommendation on user details

## A. Retrieving useful dataframe

In [4]:
Product_mapped = pd.read_csv("output/product_info_mapped.csv")
user_item_matrix = pd.read_hdf("output/user_item_matrix.hdf","user_item_matrix")
df_stand = pd.read_hdf('output/standardize_df_user_personal_info.hdf','standardize_df_user_personal_info')

## B. Doing prediction for a specific user

In [5]:
threshold = 0.99
max_neighbours = 10
pred_unknown_01, pred_known_01 = f2.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,df_non_stand,Product_mapped,user_item_matrix, verbose=True)

This len for list of neighbours for the cust_id : 266785 with a threshold of 0.99 : 

is actually 15 but we will restrict is to 10 



## C. Display result for our specific user

In [6]:
pred_unknown_01.head()

,object_name,object_id,mean_from_similar_user
14,Books_Academic,5_12,4.5
4,Footwear_Women,2_3,4.0
21,Home and kitchen_Bath,6_11,4.0
8,Electronics_Personal Appliances,3_8,3.5
1,Clothing_Women,1_1,3.5


In [7]:
pred_known_01.head()

,object_name,object_id,mean_from_similar_user,real_rating_from_266785
0,Bags_Mens,4_1,4.000000,3.0
1,Footwear_Mens,2_1,3.600000,4.0
2,Footwear_Kids,2_4,2.666667,3.5
3,Home and kitchen_Kitchen,6_10,2.500000,5.0
4,Books_Children,5_11,1.000000,5.0


## D. Information about this method

The RMSE tends toward 2.00 when we try this method on 100 differents users.

# III. Collaborative filtering memory based

## A. Retrieving useful dataframe

In [8]:
ratings_table = pd.read_csv('output/rating_table.csv')
ratings_table = ratings_table.rename(columns={"cust_id": "userId", "object_id": "ItemId", "rank": "rating", "object_name": "Item_Name"})
ratings_table = ratings_table.groupby(['userId', 'ItemId', 'Item_Name'])[['rating']].mean().reset_index()

## B. Preparing datesets

In [9]:
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user_id, item_id and ratings (in that order).
data = Dataset.load_from_df(ratings_table[['userId', 'ItemId', 'rating']], reader)
trainsetfull = data.build_full_trainset()

#we create here an anti_set
anti_set = data.build_full_trainset().build_anti_testset()

#we create here a nice_set
nice_set = data.build_full_trainset().build_testset()

## C. Train the algorithm

In [10]:
algo = KNNBasic()
algo.fit(trainsetfull)

Computing the msd similarity matrix...
Done computing similarity matrix.


## D. Display prediction

* On nice_set

In [11]:
predictions_nice = algo.test(nice_set)
all_known = pd.DataFrame(predictions_nice)
all_known.head()

,uid,iid,r_ui,est,details
0,266783,1_4,2.0,2.0,"{'actual_k': 40, 'was_impossible': False}"
1,266783,2_1,4.0,4.0,"{'actual_k': 40, 'was_impossible': False}"
2,266783,5_10,2.0,2.0,"{'actual_k': 40, 'was_impossible': False}"
3,266784,3_4,2.0,2.0,"{'actual_k': 40, 'was_impossible': False}"
4,266784,5_10,3.0,3.0,"{'actual_k': 40, 'was_impossible': False}"


* On anti_set

In [12]:
predictions_anti = algo.test(anti_set)
all_unknown = pd.DataFrame(predictions_anti)
all_unknown.head()

,uid,iid,r_ui,est,details
0,266783,3_4,2.999398,3.250000,"{'actual_k': 40, 'was_impossible': False}"
1,266783,5_7,2.999398,3.100000,"{'actual_k': 40, 'was_impossible': False}"
2,266783,2_4,2.999398,3.087500,"{'actual_k': 40, 'was_impossible': False}"
3,266783,4_1,2.999398,3.204904,"{'actual_k': 40, 'was_impossible': False}"
4,266783,5_11,2.999398,2.787500,"{'actual_k': 40, 'was_impossible': False}"


* For a specific user

In [13]:
treshold = 3.0

known = all_known[(all_known['est']>treshold)&(all_known['uid']==CURRENT_USER)].sort_values(by=['est'], ascending=False)
unknown = all_unknown[(all_unknown['est']>treshold)&(all_unknown['uid']==CURRENT_USER)].sort_values(by=['est'], ascending=False)

known.head()

,uid,iid,r_ui,est,details
9,266785,5_11,5.0,5.000000,"{'actual_k': 40, 'was_impossible': False}"
10,266785,6_10,5.0,5.000000,"{'actual_k': 40, 'was_impossible': False}"
6,266785,2_1,4.0,4.000000,"{'actual_k': 40, 'was_impossible': False}"
7,266785,2_4,3.5,3.416968,"{'actual_k': 40, 'was_impossible': False}"


In [14]:
unknown.head()

,uid,iid,r_ui,est,details
47,266785,3_8,2.999398,3.500000,"{'actual_k': 40, 'was_impossible': False}"
40,266785,1_4,2.999398,3.431074,"{'actual_k': 40, 'was_impossible': False}"
53,266785,3_9,2.999398,3.350000,"{'actual_k': 40, 'was_impossible': False}"
56,266785,5_3,2.999398,3.262500,"{'actual_k': 40, 'was_impossible': False}"
55,266785,1_1,2.999398,3.225000,"{'actual_k': 40, 'was_impossible': False}"


## E. Information about this method

In [15]:
print('On the nice_set, the RMSE score is :', accuracy.rmse(predictions_nice))

RMSE: 0.0206
On the nice_set, the RMSE score is : 0.020617845744627092


The RMSE score is lower, we have for this method 0.02

# IV. Collaborative filtering user_based

## A. Retrieving useful dataframe

In [16]:
user_item_matrix = pd.read_hdf("output/user_item_matrix.hdf","user_item_matrix")
Product_map = pd.read_csv("output/product_info_mapped.csv")

## B. Selecting a user_id and do prediction

In [17]:
#we set here the max value at 10
max_value = 10
similar_user_indices = f4.similar_users(CURRENT_USER, user_item_matrix, k=6)
pred_known_03 = f4.recommend_item_from_uid(CURRENT_USER, similar_user_indices, max_value, user_item_matrix,Product_map, True)
pred_unknown_03 = f4.recommend_item_from_uid(CURRENT_USER, similar_user_indices, max_value, user_item_matrix,Product_map, False)

## C. Display prediction for a specific user

In [18]:
pred_unknown_03.head()

,object_name,object_id,mean_from_similar_user
7,Electronics_Computers,3_5,3.5
18,Books_DIY,5_6,3.0
4,Footwear_Women,2_3,2.0
8,Electronics_Personal Appliances,3_8,1.0
0,Clothing_Mens,1_4,0.0


In [19]:
pred_known_03.head()

,object_name,object_id,mean_from_similar_user,real_rating_from_266785
0,Footwear_Kids,2_4,5.000000,3.5
1,Books_Children,5_11,3.666667,5.0
2,Home and kitchen_Kitchen,6_10,3.300000,5.0
3,Bags_Mens,4_1,3.000000,3.0
4,Footwear_Mens,2_1,2.250000,4.0


## D. Information about this method

The RMSE score tends to 1.07, meaning that the estimated ratings on average are about 1.07 higher or lower than the actual ratings.
Here, our scale is from 0 to 5, so it's is not a bit better than previous notebooks.

# V. Create the consolidated Data frame

## A. Set up user

In [20]:
CURRENT_USER = 266794

index = False
cust_id = CURRENT_USER

customer_index, customer_id = f2.select_customer(index , cust_id, df_non_stand)
print('index:', customer_index)
print('Id:', customer_id)

index: 4
Id: 266794


## B. Result for model_01

In [21]:
threshold = 0.99
max_neighbours = 10

pred_unknown_01, pred_known_01 = f2.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,df_non_stand,Product_mapped,user_item_matrix, verbose = True)

This len for list of neighbours for the cust_id : 266794 with a threshold of 0.99 : 

is actually 11 but we will restrict is to 10 



## C. Result for model_02

In [22]:
#treshold at 0 because we want all value here
treshold = 0

pred_known_02 = all_known[(all_known['est']>treshold)&(all_known['uid']==CURRENT_USER)].sort_values(by=['est'], ascending=False)
pred_unknown_02 = all_unknown[(all_unknown['est']>treshold)&(all_unknown['uid']==CURRENT_USER)].sort_values(by=['est'], ascending=False)

## D. Result for model_03

In [23]:
max_value = 10

similar_user_indices = f4.similar_users(CURRENT_USER, user_item_matrix, k=10)
pred_known_03 = f4.recommend_item_from_uid(CURRENT_USER, similar_user_indices, max_value, user_item_matrix,Product_map, True)
pred_unknown_03 = f4.recommend_item_from_uid(CURRENT_USER, similar_user_indices, max_value, user_item_matrix,Product_map, False)

## E. Merging all known values for this user

We now have 3 differents models, we can use weights to give more importance to one over the other.  
We use A, B, and C as weight for respectively model 01, 02, 03.  

For a reminder :
 - Model 01 is based on similarity over personal information
 - Model 02 is based on collaborative filtering using suprise library
 - Model 03 is based on collaborative filtering too but take cosine similarity between users and use the mean of unrating item to estimate unknown ratings

In [24]:
A,B,C = 1,1,1
consolidated_known = f5.consolidate_function(pred_known_01,pred_known_02, pred_known_03, A,B,C, known = True)

In [25]:
consolidated_known

,object_name,object_id,real_rank,pred_01,pred_02,pred02_worked,pred_03,consolidated_estimation
0,Bags_Women,4_4,3.0,4.000000,3.000000,True,3.400000,3.466667
1,Books_Academic,5_12,4.0,3.666667,4.000000,True,3.722222,3.796296
2,Clothing_Kids,1_3,3.5,3.333333,3.574694,True,3.357143,3.421723
3,Footwear_Women,2_3,2.5,3.000000,2.495468,True,4.333333,3.276267
4,Bags_Mens,4_1,2.0,3.000000,2.000000,True,2.500000,2.500000
5,Books_Children,5_11,3.0,3.000000,3.000000,True,2.400000,2.800000
6,Electronics_Personal Appliances,3_8,4.0,2.333333,4.000000,True,3.500000,3.277778
7,Electronics_Audio and video,3_10,1.5,0.000000,1.577476,True,2.500000,2.038738


When all weights A, B, C are set to 1, we can easily see that the prediction 2 is performing well better than other.  
It will be logic that B should be higher than A and C.

## E. Finding the best way to consolidated all the metrics by running a loop over user

In [26]:
def find_metrics(specific_user):
    
    #finding predictions for model_01 :     
    customer_index, customer_id = f2.select_customer(index , specific_user, df_non_stand)
    _, pred_known_01 = f2.do_prediction(customer_index, customer_id, max_neighbours, threshold,df_stand,
                                                      df_non_stand,Product_mapped,user_item_matrix, verbose = False)

    #finding predictions for model_02 : 
    pred_known_02 = all_known[(all_known['est']>treshold)&(all_known['uid']==customer_id)].sort_values(by=['est'], ascending=False)
     
    #finding predictions for model_03 : 
    similar_user_indices = f4.similar_users(specific_user, user_item_matrix, k=6)
    pred_known_03 = f4.recommend_item_from_uid(specific_user, similar_user_indices, max_value, user_item_matrix,Product_map, True)
       
    consolidated_known = f5.consolidate_function(pred_known_01,pred_known_02, pred_known_03, A,B,C, known = True)
    mse, rmse = f5.meatrics_mse_rmse_consolidated(consolidated_known)
    MSE.append(mse)
    RMSE.append(rmse)

In [27]:
RMSE = []
MSE = []
list_user = ratings_table.userId.unique().tolist()

t1 = time.time()

df_result = pd.DataFrame(columns=['A','B','C','mean_mse', 'mean_rmse']) 

for A in [1,2]:
    for B in [3,4]:
        for C in [1,2]:
            for i, usr in enumerate(list_user):
                find_metrics(usr)

                if i == 15: # we stop when we try it for 15 differents users
                    
                    L = [[A,B,C,statistics.mean(MSE), statistics.mean(RMSE)]]
                    RMSE = []
                    MSE = []
                    
                    df_result = df_result.append(pd.DataFrame(L,columns=['A','B','C','mean_mse', 'mean_rmse'] ))
                    
                    break
                    
t2 = time.time()

print('took in sec :', (t2 - t1))   

result_ABC = df_result.sort_values(by=['mean_rmse'], ascending = True)
result_ABC.to_csv('output/result_ABC.csv')
result_ABC.head(20)

took in sec : 134.1696081161499


,A,B,C,mean_mse,mean_rmse
0,1,4,1,0.180883,0.396553
0,1,4,2,0.247168,0.464264
0,1,3,1,0.262168,0.478023
0,1,3,2,0.339432,0.544748
0,2,4,1,0.373752,0.568178
0,2,4,2,0.414349,0.602238
0,2,3,1,0.511603,0.665883
0,2,3,2,0.546415,0.692733


Now that we find a RMSE at 0.396 we can keep this set of value A,B,C at 1,4,1.   
Meaning that we accord much more importance at B so the method 02 and less at method 01 and 03.  
That is quite logic, because rmse for model 02 is close to 0.11. 
Here, the best model should be the one with B at 1 , A and C at 0.  
But I want to conserve all those 3 models so, I user 1,4,1

## F. Merging all unknown values for our specific user

In [28]:
A,B,C = 1,4,1

In [29]:
final_prediction = f5.consolidate_function(pred_unknown_01,pred_unknown_02, pred_unknown_03, A,B,C, known = False).sort_values(by=['consolidated_estimation'], ascending = False)
final_prediction

,object_name,object_id,rank_by_default,pred_01,pred_02,pred02_worked,pred_03,consolidated_estimation
0,Home and kitchen_Bath,6_11,2.999398,5.000000,3.336921,True,0.0,3.669537
1,Electronics_Computers,3_5,2.999398,4.666667,2.932011,True,2.0,3.065785
5,Home and kitchen_Furnishing,6_2,2.999398,4.000000,3.200000,True,1.0,2.966667
2,Books_DIY,5_6,2.999398,4.666667,2.525000,True,0.0,2.953333
8,Clothing_Mens,1_4,2.999398,3.000000,2.924156,True,3.0,2.949437
9,Electronics_Mobiles,3_4,2.999398,2.666667,2.987500,True,0.0,2.923333
7,Books_Non-Fiction,5_10,2.999398,3.666667,2.712500,True,3.0,2.919444
3,Books_Comics,5_3,2.999398,4.333333,2.425000,True,3.0,2.838889
4,Books_Fiction,5_7,2.999398,4.000000,2.888913,True,1.0,2.759275
6,Clothing_Women,1_1,2.999398,3.750000,2.512500,True,2.0,2.633333


So, we finnaly obtain our prediction for our specific customer.  
In the next cell we can see the 3 best option for himm according to my choice ;)

In [30]:
print(f'The Top 5 best prediction for user_id {CURRENT_USER} are : \n')
final_prediction.head(5)

The Top 5 best prediction for user_id 266794 are : 



,object_name,object_id,rank_by_default,pred_01,pred_02,pred02_worked,pred_03,consolidated_estimation
0,Home and kitchen_Bath,6_11,2.999398,5.000000,3.336921,True,0.0,3.669537
1,Electronics_Computers,3_5,2.999398,4.666667,2.932011,True,2.0,3.065785
5,Home and kitchen_Furnishing,6_2,2.999398,4.000000,3.200000,True,1.0,2.966667
2,Books_DIY,5_6,2.999398,4.666667,2.525000,True,0.0,2.953333
8,Clothing_Mens,1_4,2.999398,3.000000,2.924156,True,3.0,2.949437


## This is the end of the fifth part and final part.
## I hope you enjoy this reading